<a href="https://colab.research.google.com/github/AslanDevbrat/NLP/blob/main/Neural_Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATASET

In [26]:
!wget http://manythings.org/anki/fra-eng.zip

--2022-07-19 12:26:52--  http://manythings.org/anki/fra-eng.zip
Resolving manythings.org (manythings.org)... 173.254.30.110
Connecting to manythings.org (manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.manythings.org/anki/fra-eng.zip [following]
--2022-07-19 12:26:52--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Reusing existing connection to manythings.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 6612164 (6.3M) [application/zip]
Saving to: ‘fra-eng.zip.1’

fra-eng.zip.1       100%[===================>]   6.31M  3.70MB/s    in 1.7s    

2022-07-19 12:26:54 (3.70 MB/s) - ‘fra-eng.zip.1’ saved [6612164/6612164]



In [27]:
!unzip /content/fra-eng.zip

Archive:  /content/fra-eng.zip
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: _about.txt              
  inflating: fra.txt                 


In [69]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.layers import SimpleRNN,LSTM, Dense, Reshape, Embedding,GRU,Bidirectional, Input
import numpy as np
import re
import string
import nltk
import numpy as np
from matplotlib import pyplot as plt
import pandas
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [223]:
path= "/content/fra.txt"
NUM_EXAMPLES = 2500
text_dataset = tf.data.TextLineDataset(path).take(NUM_EXAMPLES)


In [224]:
def selector(input_text):
  return tf.strings.split(input_text, '\t')[0:1], 'starttoken' + tf.strings.split(input_text, '\t')[1:2], tf.strings.split(input_text, '\t')[1:2]

text_dataset = text_dataset.map(selector)


In [225]:


for i in text_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttokenVa !'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va !'], dtype=object)>)


In [226]:
def preprocess_sentences(input_data):
    '''
    Input: raw reviews
    output: standardized reviews
    '''
    output=tf.strings.lower(input_data)
    outputs=tf.strings.regex_replace(output,"<[^>]+>","")
    outputs=tf.strings.regex_replace(output,"<[%s]"%re.escape(string.punctuation)," ")
    outputs=tf.strings.regex_replace(output,"  "," ")
    
    return output
SEQUENCE_LENGTH = 10
vectorize_input_layer = TextVectorization(
      standardize = preprocess_sentences,
      output_sequence_length = SEQUENCE_LENGTH,

      )
      
vectorize_output_layer = TextVectorization(
    standardize = preprocess_sentences,
    output_sequence_length = SEQUENCE_LENGTH,
)

vectorize_pre_output_layer = TextVectorization(
    standardize = preprocess_sentences,
    output_sequence_length = SEQUENCE_LENGTH,
)

In [227]:
training_data = text_dataset.map(lambda x,y, z :x)
vectorize_input_layer.adapt(training_data)

training_data = text_dataset.map(lambda x, y, z :y)
vectorize_pre_output_layer.adapt(training_data)

training_data = text_dataset.map(lambda x, y, z :z)
vectorize_output_layer.adapt(training_data)


In [228]:
VOCAB_INPUT_SIZE  = len(vectorize_input_layer.get_vocabulary())
VOCAB_OUTPUT_SIZE = len(vectorize_output_layer.get_vocabulary())
VOCAB_PRE_OUTPUT_SIZE=len(vectorize_pre_output_layer.get_vocabulary())
print(VOCAB_OUTPUT_SIZE)

1830


In [229]:


def vectorizer(x, y, z):
  return {'in1': tf.squeeze(vectorize_input_layer(x), 0) , 'in2': tf.squeeze(vectorize_pre_output_layer(y),0) }, tf.squeeze(tf.one_hot(vectorize_output_layer(z), depth = VOCAB_OUTPUT_SIZE), 0)

dataset = text_dataset.map(vectorizer)

In [230]:
for i in dataset.take(1):

  print(i)

({'in1': <tf.Tensor: shape=(10,), dtype=int64, numpy=array([24,  0,  0,  0,  0,  0,  0,  0,  0,  0])>, 'in2': <tf.Tensor: shape=(10,), dtype=int64, numpy=array([37,  2,  0,  0,  0,  0,  0,  0,  0,  0])>}, <tf.Tensor: shape=(10, 1830), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)


In [231]:
VALIDATION_RATIO = 1
VALIDATION_BRIDGE = int(VALIDATION_RATIO* NUM_EXAMPLES)
dataset = dataset.shuffle(NUM_EXAMPLES)
train_dataset = dataset.take(VALIDATION_BRIDGE)
validation_dataset = dataset.skip(VALIDATION_BRIDGE)

In [232]:
BATCH_SIZE = 64
train_dataset=train_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset=validation_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# MODELLING

In [233]:
SEQUENCE_LENGTH = 10
EMBEDDING_DIM = 8
enc_size = 300
dec_size = 100


enc_input = Input((SEQUENCE_LENGTH,),name = 'in1')
dec_input = Input((SEQUENCE_LENGTH,), name = 'in2')

x = Embedding(VOCAB_INPUT_SIZE, EMBEDDING_DIM)(enc_input)

encoder = LSTM(enc_size, return_sequences= False, return_state = True)
_ , h, c = encoder(x)

x = Embedding(VOCAB_PRE_OUTPUT_SIZE, EMBEDDING_DIM)(dec_input)

decoder = LSTM(dec_size, return_sequences=True, return_state = True)
h = Dense(dec_size)(h)
c = Dense(dec_size)(c)

x, h, c = decoder(x, [h, c])
x = Dense(VOCAB_OUTPUT_SIZE, activation = "softmax")(x)

model = tf.keras.Model([enc_input, dec_input], x)
model.summary()





Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in1 (InputLayer)               [(None, 10)]         0           []                               
                                                                                                  
 embedding_54 (Embedding)       (None, 10, 8)        6768        ['in1[0][0]']                    
                                                                                                  
 in2 (InputLayer)               [(None, 10)]         0           []                               
                                                                                                  
 lstm_49 (LSTM)                 [(None, 300),        370800      ['embedding_54[0][0]']           
                                 (None, 300),                                              

In [234]:
LR = 1e-3
EPOCH = 100

model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(lr = LR,),
    metrics = ["accuracy"],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [235]:
for x in train_dataset.take(1):
  print(x)

({'in1': <tf.Tensor: shape=(64, 10), dtype=int64, numpy=
array([[  6, 362,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  74,  60,   0,   0,   0,   0,   0,   0,   0],
       [ 14,  37,   0,   0,   0,   0,   0,   0,   0,   0],
       [475, 142,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  58, 139,   0,   0,   0,   0,   0,   0,   0],
       [ 15,  40,  24,   0,   0,   0,   0,   0,   0,   0],
       [416,   4,   0,   0,   0,   0,   0,   0,   0,   0],
       [  6,  31,   0,   0,   0,   0,   0,   0,   0,   0],
       [238,  19,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 58, 130,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  72,   4,   0,   0,   0,   0,   0,   0,   0],
       [  2,  76,   4,   0,   0,   0,   0,   0,   0,   0],
       [  5, 103,   0,   0,   0,   0,   0,   0,   0,   0],
       [  8, 134,   0,   0,   0,   0,   0,   0,   0,   0],
       [189,  75,   0,   0,   0,   0,   0,   0,   0,   0],
       [409,  12,   0,   0,   0,   0,   0,   0,   0,   0],

In [ ]:
checkpoint_filepath  = "seq2seq.hdf5"

#checkpoint_filepath="imdb/basic_rnn.hdf5"
callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_filepath,### WHERE BEST WEIGHTS ARE STORED
    save_weights_only=True,
   monitor="loss",
   mode='min',
    save_best_only=True)
#Train model on dataset
history=model.fit(train_dataset,verbose=1,epochs=EPOCH,callbacks=[callback])#

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


In [ ]:
 vectorize_output_layer.get_vocabulary()